In [22]:
import warnings
import json
import os
import openai
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

warnings.filterwarnings("ignore", category=DeprecationWarning, module="langchain")
openai.api_key = os.environ.get("MY_OPENAI_API_KEY")

In [23]:
def process_json_data(json_files):
    """
    여러 JSON 파일을 읽고 데이터를 통합한 후 특정 형식의 문자열 리스트로 반환

    Parameters:
        json_files (list): JSON 파일 경로 리스트

    Returns:
        list: 파일 데이터에서 'title'과 'content'를 읽어 특정 형식으로 변환한 리스트
    """
    all_json_data = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_json_data.extend(data)
        except FileNotFoundError:
            print(f"Error: 파일을 찾을 수 없습니다 - {file_path}")
        except json.JSONDecodeError:
            print(f"Error: JSON 파일 형식이 잘못되었습니다 - {file_path}")

    # 'title'과 'content'를 읽어 특정 형식으로 반환
    return [
        f"Title: {item.get('title', 'N/A')}\nContent: {item.get('content', 'N/A')}"
        for item in all_json_data
    ]

def title_json_data(json_files):
    """
    여러 JSON 파일을 읽고 데이터를 통합한 후 특정 형식의 문자열 리스트로 반환

    Parameters:
        json_files (list): JSON 파일 경로 리스트

    Returns:
        list: 파일 데이터에서 'title'과 'content'를 읽어 특정 형식으로 변환한 리스트
    """
    all_json_data = []
    for file_path in json_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                all_json_data.extend(data)
        except FileNotFoundError:
            print(f"Error: 파일을 찾을 수 없습니다 - {file_path}")
        except json.JSONDecodeError:
            print(f"Error: JSON 파일 형식이 잘못되었습니다 - {file_path}")

    # 'title'과 'content'를 읽어 특정 형식으로 반환
    title = [item.get('title', 'N/A') for item in all_json_data]
    return title

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def split_texts(texts, chunk_size=1000, chunk_overlap=200):
    """
    텍스트 데이터를 입력 받아 분할

    Parameters:
        texts: 텍스트 데이터

    Returns:
        Document 객체
    """
    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )
    documents = [Document(page_content=texts)]
    return recursive_text_splitter.split_documents(documents)

In [25]:
from langchain.memory import ConversationSummaryMemory

def documents_filter(SPLITS):
    """
    분할된 데이터에서 불필요한 데이터를 제거하고 하나로 결합
    ConversationSummaryMemory에 이전 내용을 요약하여 저장
    아전 내용과 대조해서 불필요한 데이터 구분

    Parameters:
        SPLITS: 분할된 텍스트 데이터 : Document

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
                model="gpt-4o-mini",
                api_key=openai.api_key,
                max_tokens=1000,
                temperature=0.0,
            )
    summaries = []
    memory = ConversationSummaryMemory(
        llm=llm, return_messages=True)
    
    count = 0
    for SPLIT in SPLITS:
        SPLIT = SPLIT.page_content

        try:
            context = memory.load_memory_variables({})["history"]
            prompt = ChatPromptTemplate.from_template(
                """
                persona : documents filter
                language : only in korean
                extract the parts related to the context and ignore the rest,
                write blanck if it's not relevant,
                
                <context>
                {context}
                </context>
                
                <docs>
                {SPLIT}
                </docs>
                """
            )
            chain = prompt | llm | StrOutputParser()
            summary = chain.invoke({"SPLIT": SPLIT, 'context' : context})
            memory.save_context({"input": f'summary # {count}'}, {"output": summary})
            summaries.append(summary)
            count+=1

        except Exception as e:
            # 오류 처리: 만약 API 호출 중에 문제가 발생하면 오류 메시지 추가
            print(f"Error summarizing document: {e}")
            summaries.append(f"Error summarizing document: {e}")

    return "".join(summaries)

In [26]:
def generate_script(summaries):
    """
    대화 LLM이 전달할 이야기의 대본 생성

    Parameters:
        summaries: 필터링된 텍스트 데이터

    Returns:
        텍스트 데이터
    """
    llm = ChatOpenAI(
        model="gpt-4o-mini",
        api_key=openai.api_key,
        max_tokens=5000,
        temperature=0.0,
    )
    prompt = ChatPromptTemplate.from_template(
    """
    persona = script writer
    language = only in korean
    least 3000 tokens
    use input,
    refer to sample,
    write about time, character, event,
    write only fact
    ignore the mere listing of facts and write N/A
 
    <sample>
    # title : title of script
    # prologue 1 : song, movie, book, show about subject
    - coontent :
    # prologue 2 : explain about subject
    - coontent :
    # prologue 3 : explain about character
    - coontent :
    # exposition 1 : historical background of subject
    - coontent :
    # exposition 2 : history of character
    - coontent :
    # exposition 3 : beginning of event
    - coontent :
    # development 1 : situation, action, static of character
    - coontent :
    # development 2 : influence of event
    - coontent :
    # development 3 : reaction of people
    - coontent :
    # climax 1 : event and effect bigger
    - coontent :
    # climax 2 : dramatic action, conflict
    - coontent :
    # climax 3 : falling Action
    - coontent :
    # denouement : resolution
    - coontent :
    # epilogue : message, remaining
    - coontent :
    </sample>

    <input>
    {summaries}
    </input>

    """
    )
    chain = prompt | llm | StrOutputParser()
    script = chain.invoke({"summaries": summaries})
    return script

In [27]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

def create_vector_store(DOCS, db_name: str, DB_PATH):
    return Chroma.from_documents(
        documents=DOCS,
        collection_name=db_name,
        persist_directory=DB_PATH,
        embedding=OpenAIEmbeddings(
            model="text-embedding-ada-002", api_key=openai.api_key
        ),
    )

In [28]:
json_files = ['./documents/filtered_unsolved_cases.json', './documents/korea_crime.json']
crimes = process_json_data(json_files)
titles = title_json_data(json_files)

In [29]:
sample_crimes = crimes[0:11]
sample_titles = titles[0:11]

In [37]:
sample_crimes

['Title: 정인숙 피살사건\nContent: 정인숙 피살사건(鄭仁淑 被殺事件)은제3공화국당시인1970년3월 17일밤 11시경,대한민국서울특별시마포구합정동절두산근처 도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인정인숙이 사망한 사건이다.[1][2]피해자 정인숙(본명: 정금지)은 총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다.\n정인숙은 당시 출산한 아이의 아버지인대한민국의 국무총리정일권과 갈등관계에 있었고,신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 제기했으나 유야무야 묻혀져 아직까지의문사로 남아있다.\n경과[편집]\n경과\n1970년3월 17일밤 11시경, 서울마포구합정동절두산 부근의강변3로에 멈춰 서 있는검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는정종욱(당시 34세), 숨진 여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다.\n당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다.[3][4]아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다.\n고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다.[4]\n나중에 정인숙의 집에서 발견된 정인숙의 소지품에선 정관계 고위층의 명함 26장이 포함된 33장의 명함이 쏟아져 나왔는데, 명단에는박정희,정일권,이후락,김형욱등 대다수 5.16 주체세력들이  포함되어 있었다.[5]이후 경찰 수사는 지지부진해졌고 언론 보도가 수사를 대신하게 되었다. 언론은 정인숙에게 숨겨진 아들(정일성,정성일, 또는박승일(68년 3월생)[6])이 하나 있고, 그 아들은 당시 청와대 고위층의 자녀라는 소문과 정인숙이 당시 정관계 고위층 전용이라 할 

In [38]:
sample_titles

['정인숙 피살사건',
 '춘천 강간살인 조작 사건',
 '부산 어린이 연쇄살인 사건',
 '대한항공 007편 격추 사건',
 '신호수 의문사 사건',
 '김훈 중위 사건',
 '이춘재 연쇄 살인 사건',
 '개구리 소년 사건',
 '이형호 유괴 살해 사건',
 '치과의사 모녀 살인 사건',
 '대구 아동 황산테러 사건']

In [39]:
def script_maker(DOCS : str):
  splits = split_texts(DOCS)
  return documents_filter(splits)

In [40]:
splits = [script_maker(sample) for sample in sample_crimes]

In [41]:
splits

['<context>\n[SystemMessage(content=\'\', additional_kwargs={}, response_metadata={})]\n</context>\n\n<docs>\nTitle: 정인숙 피살사건\nContent: 정인숙 피살사건(鄭仁淑 被殺事件)은 1970년 3월 17일 밤 11시경, 대한민국 서울특별시 마포구 합정동 절두산 근처 도로에서 발생한 교통사고를 가장한 총격 살인사건으로 고급 요정 종업원인 정인숙이 사망한 사건이다. 피해자 정인숙(본명: 정금지)은 총상으로 요절하였고 그의 차를 운전하던 넷째 오빠 정종욱은 넓적다리에 관통상을 입었으나 지나가던 택시기사에게 도움을 청하여 구조되었다. 정인숙은 당시 출산한 아이의 아버지인 대한민국의 국무총리 정일권과 갈등관계에 있었고, 신민당은 이 사건의 배후로 정부 고위층의 개입 의혹을 제기했으나 유야무야 묻혀져 아직까지 의문사로 남아있다. 1970년 3월 17일 밤 11시경, 서울 마포구 합정동 절두산 부근의 강변 3로에 멈춰 서 있는 검은색 코로나 승용차에서 권총에 넓적다리를 관통당해 신음하고 있는 한 사내와, 머리와 가슴에 총을 맞아 이미 숨진 한 젊은 여인이 발견되었다. 부상당한 사내는 정종욱(당시 34세), 숨진 여인은 정인숙(당시 25세)으로 두 사람은 남매 관계로 밝혀졌다. 당시 26세였던 정인숙에게는 3살 된 아들이 1명 있는 것으로 확인되었다. 아이의 아버지에 대해서는 당시 정부의 한 유력인사가 지목되기도 했다. 고위 공무원의 딸로 태어나 대학을 중퇴하였으며 여러 명의 친오빠가 있었다. 그 뒤 정인숙은 당시 한일회담도 이루어진 선운각 등 최고급 요정에서 호스티스로 일하고 있었다.\n</docs>정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 살인 사건이다. 피해자인 정인숙은 고급 호스티스로 일하던 중 총에 맞아 사망하였고, 그녀의 오빠 정종욱은 부상을 입고 살아남았다. 사건은 정인숙의 자녀 아버지가 유력 정치인이라는 점에서 정부의 개입 의혹을 불

In [42]:
scripts = [generate_script(split) for split in splits]

In [43]:
scripts

['# 제목: 정인숙 피살사건\n\n# 프롤로그 1: 사건을 다룬 미디어\n- 2010년 3월 20일, SBS TV에서 방영된 프로그램은 정인숙 피살사건의 실체를 추적하는 내용을 담고 있다. 이 프로그램은 사건의 복잡성과 미해결 상태를 조명하며, 당시 수사 기록과 현장 감식 자료를 공개하여 새로운 시각을 제공하였다.\n\n# 프롤로그 2: 사건의 배경\n- 정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 살인 사건이다. 이 사건은 고급 호스티스에서 일하던 정인숙이 총에 맞아 사망한 사건으로, 그녀의 형 정종욱이 부상을 입고 생존하였다. 사건은 정인숙의 자녀 아버지가 고위 정치인이라는 점에서 정부의 개입 의혹을 불러일으켰다.\n\n# 프롤로그 3: 정인숙의 인물상\n- 정인숙은 1945년 1월 1일에 태어나 고위 공직자 가문에서 자랐다. 그녀는 대학을 중퇴하고 고급 호스티스로 일하며 사회의 주목을 받았다. 정인숙은 당시 26세였고, 3세 아들을 두고 있었다. 그녀의 소지품에서 고위 인사들의 명함이 발견되었고, 숨겨진 자녀에 대한 소문이 돌았다.\n\n# 전개 1: 사건의 역사적 배경\n- 1970년대 초반, 한국은 정치적 불안정과 사회적 갈등이 심화되던 시기였다. 이 시기에 발생한 정인숙 피살사건은 고위층의 부패와 권력 남용에 대한 의혹을 불러일으켰다. 사건은 당시 사회의 불신과 불안감을 더욱 부각시켰다.\n\n# 전개 2: 정인숙의 개인사\n- 정인숙은 고위 공무원의 딸로 태어나 여러 명의 친오빠가 있었다. 그녀는 고급 요정에서 호스티스로 일하며, 당시 한일회담이 이루어진 선운각 등에서 활동하였다. 정인숙은 당시 정부의 유력 인사와의 관계로 인해 사회적 스캔들의 중심에 서게 되었다.\n\n# 전개 3: 사건의 시작\n- 1970년 3월 17일 밤 11시경, 서울 마포구 합정동 절두산 근처 도로에서 정인숙과 그녀의 형 정종욱이 탄 검은색 코로나 승용차가 멈춰 서 있었다. 이때 정인숙은 총에 맞아 사망하고, 정종욱은 부상을 입었다. 사건

In [44]:
new_scripts = [Document(page_content=script) for script in scripts]

In [45]:
new_scripts

[Document(metadata={}, page_content='# 제목: 정인숙 피살사건\n\n# 프롤로그 1: 사건을 다룬 미디어\n- 2010년 3월 20일, SBS TV에서 방영된 프로그램은 정인숙 피살사건의 실체를 추적하는 내용을 담고 있다. 이 프로그램은 사건의 복잡성과 미해결 상태를 조명하며, 당시 수사 기록과 현장 감식 자료를 공개하여 새로운 시각을 제공하였다.\n\n# 프롤로그 2: 사건의 배경\n- 정인숙 피살사건은 1970년 3월 17일 서울에서 발생한 교통사고로 위장된 살인 사건이다. 이 사건은 고급 호스티스에서 일하던 정인숙이 총에 맞아 사망한 사건으로, 그녀의 형 정종욱이 부상을 입고 생존하였다. 사건은 정인숙의 자녀 아버지가 고위 정치인이라는 점에서 정부의 개입 의혹을 불러일으켰다.\n\n# 프롤로그 3: 정인숙의 인물상\n- 정인숙은 1945년 1월 1일에 태어나 고위 공직자 가문에서 자랐다. 그녀는 대학을 중퇴하고 고급 호스티스로 일하며 사회의 주목을 받았다. 정인숙은 당시 26세였고, 3세 아들을 두고 있었다. 그녀의 소지품에서 고위 인사들의 명함이 발견되었고, 숨겨진 자녀에 대한 소문이 돌았다.\n\n# 전개 1: 사건의 역사적 배경\n- 1970년대 초반, 한국은 정치적 불안정과 사회적 갈등이 심화되던 시기였다. 이 시기에 발생한 정인숙 피살사건은 고위층의 부패와 권력 남용에 대한 의혹을 불러일으켰다. 사건은 당시 사회의 불신과 불안감을 더욱 부각시켰다.\n\n# 전개 2: 정인숙의 개인사\n- 정인숙은 고위 공무원의 딸로 태어나 여러 명의 친오빠가 있었다. 그녀는 고급 요정에서 호스티스로 일하며, 당시 한일회담이 이루어진 선운각 등에서 활동하였다. 정인숙은 당시 정부의 유력 인사와의 관계로 인해 사회적 스캔들의 중심에 서게 되었다.\n\n# 전개 3: 사건의 시작\n- 1970년 3월 17일 밤 11시경, 서울 마포구 합정동 절두산 근처 도로에서 정인숙과 그녀의 형 정종욱이 탄 검은색 코로나 승용차가 멈춰 서 있었다. 이

In [46]:
path = './db/script_db'
db_name = 'script_db'
script_db = create_vector_store(new_scripts, db_name, path)